# Homework: Decision Tree
## Phase 1: Implementing the ID3 Decision Tree Algorithm
***

## Import Libraries

In [2]:
# complete
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Load Dataset (CSV file)

In [31]:
# complete
train_data_m = pd.read_csv("PlayTennis_train.csv")
test_data_m = pd.read_csv("PlayTennis_test.csv")
train_data_m.shape[0]
train_data_m.shape[1]
instancesNum = train_data_m.shape[0]
featuresNum = train_data_m.shape[1]
labelClassNum = len(np.unique(train_data_m['Play Tennis']))
print(f"Number of rows is {instancesNum} & Number of columns is {featuresNum} & Number of features is {featuresNum-1} & Number of classes/labels is {labelClassNum}.")

Number of rows is 14 & Number of columns is 5 & Number of features is 4 & Number of classes/labels is 2.


In [4]:
train_data_m.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


## Calculating the entropy of the whole dataset

In [5]:
def calc_total_entropy(train_data, label, class_list):
    # complete
    totalSize = train_data.shape[0]
    totalEntropy = 0
    # each class of the label
    for c in class_list:
        totalClassNum = train_data[train_data[label] == c].shape[0]
        # entropy of each class
        totalClassEntropy = - (totalClassNum/totalSize)*np.log2(totalClassNum/totalSize)
        # calc. total entropy of dataset
        totalEntropy += totalClassEntropy
    return totalEntropy

## Calculating the entropy for a feature

In [6]:
def calc_entropy(feature_value_data, label, class_list):
    # complete
    classNum = feature_value_data.shape[0]
    entropy = 0
    for c in class_list:
        # size of a class
        labelClassNum = feature_value_data[feature_value_data[label] == c].shape[0]
        classEntropy = 0
        if labelClassNum != 0:
            probability = labelClassNum/classNum
            # entropy
            classEntropy = - probability * np.log2(probability)
        # calc. total entropy of class
        entropy += classEntropy
    return entropy

## Calculating information gain for a feature

In [7]:
def calc_info_gain(feature_name, train_data, label, class_list):
    # complete
    fValueList = train_data[feature_name].unique() #unqiue values of the feature
    totalSize = train_data.shape[0]
    featureIG = 0.0
    for fvalue in fValueList:
        # just feature value rows
        feature_value_data = train_data[train_data[feature_name] == fvalue]
        fValueNum = feature_value_data.shape[0]
        # entropy of each feature value
        fValueEntropy = calc_entropy(feature_value_data, label, class_list)
        fValueProb = fValueNum / totalSize
        # calc. information gain of a feature value
        featureIG += fValueProb * fValueEntropy
    
    # return information gain   
    return calc_total_entropy(train_data, label, class_list) - featureIG


## Finding the most informative feature (feature with highest information gain)

In [8]:
de(train_data, label, class_list):
    # complete
    # extract feature names
    feature_list = train_data.columns.drop(label)
    maxIG = -1
    maxIGfeature = None
    
    for f in feature_list:
        # calc. IG for each feature
        featureIG = calc_info_gain(f, train_data, label, class_list)
        # finding MAX information gain
        if maxIG < featureIG:
            maxIG = featureIG
            maxIGfeature = f
            
    return maxIGfeature

## Adding a node to the tree

In [9]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
    tree = {}
    
    for feature_value, count in feature_value_count_dict.items():
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        
        assigned_to_node = False
        for c in class_list:
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]

            if class_count == count:
                tree[feature_value] = c
                train_data = train_data[train_data[feature_name] != feature_value]
                assigned_to_node = True
        if not assigned_to_node:
            tree[feature_value] = "?"
            
    return tree, train_data

##  Performing ID3 Algorithm and generating Tree

In [10]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    # complete
    # check if dataset is not empty
    if train_data.shape[0] != 0:
        # find next feature with maxIG to be a node
        maxIGfeature = find_most_informative_feature(train_data, label, class_list)
        # generate sub-tree and updated dataset
        tree, train_data = generate_sub_tree(maxIGfeature, train_data, label, class_list)
        nodes = None
        # if we are not at the root of tree add a feature to nodes
        if prev_feature_value != None:
            root[prev_feature_value] = dict()
            root[prev_feature_value][maxIGfeature] = tree
            nodes = root[prev_feature_value][maxIGfeature]
        else: # add it as root of the tree
            root[maxIGfeature] = tree
            nodes = root[maxIGfeature]
        
        # check each nodes
        for node, branch in list(nodes.items()):
            # if it is not a leaf
            if branch == "?":
                # change feature_value_data with updated dataset to expand the node
                feature_value_data = train_data[train_data[maxIGfeature] == node]
                # call func. recursively
                make_tree(nodes, node, feature_value_data, label, class_list)


## Finding unique classes of the label and Starting the algorithm

In [11]:
def id3(train_data_m, label):
    train_data = train_data_m.copy()
    tree = {}
    class_list = train_data[label].unique()
    make_tree(tree, None, train_data, label, class_list)
    
    return tree

## Predicting from the tree

In [12]:
def predict(tree, instance):
    if not isinstance(tree, dict):
        return tree
    else:
        root_node = next(iter(tree))
        feature_value = instance[root_node]
        if feature_value in tree[root_node]:
            return predict(tree[root_node][feature_value], instance)
        else:
            return None

## Evaluating test dataset

In [13]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows():
        result = predict(tree, test_data_m.iloc[index])
        if result == test_data_m[label].iloc[index]:
            correct_preditct += 1
        else:
            wrong_preditct += 1
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)
    return accuracy

In [14]:
tree = id3(train_data_m, 'Play Tennis')
tree

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [15]:
accuracy = evaluate(tree, test_data_m, 'Play Tennis')
print("accuracy:", accuracy)

accuracy: 1.0


## Pruning

In [16]:
# def rep_pruning(tree, validation_data, label):
# complete
train_data = pd.read_csv("mushrooms_train.csv")
valid_data = pd.read_csv("mushrooms_valid.csv")
test_data = pd.read_csv("mushrooms_test.csv")    

tree = id3(train_data, 'poisonous')
acc = evaluate(tree, train_data, 'poisonous')
print("training accuracy: ",acc)
acc = evaluate(tree, valid_data, 'poisonous')
print(f"validation accuracy: {acc:.3f}")
acc = evaluate(tree, test_data, 'poisonous')
print(f"test accuracy: {acc:.3f}")

training accuracy:  1.0
validation accuracy: 0.787
test accuracy: 0.796
